### Build `PerspectiveTool`

In [ ]:
%pip install google-api-python-client -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip freeze | grep llama-index

llama-index-agent-openai==0.2.1
llama-index-callbacks-arize-phoenix==0.1.4
-e git+ssh://git@github.com/run-llama/llama_index.git@0422270a4324247f9cc98a84846869030a52d4da#egg=llama_index_core&subdirectory=llama-index-core
llama-index-embeddings-openai==0.1.6
llama-index-llms-openai==0.1.12
llama-index-readers-file==0.1.6
openinference-instrumentation-llama-index==1.1.1


In [ ]:
from googleapiclient import discovery
from typing import Dict, Optional
import json
import os


class Perspective:
    """Custom class to interact with Perspective API."""

    attributes = [
        "toxicity",
        "severe_toxicity",
        "identity_attack",
        "insult",
        "profanity",
        "threat",
        "sexually_explicit",
    ]

    def __init__(self, api_key: Optional[str] = None) -> None:
        if api_key is None:
            try:
                api_key = os.environ["PERSPECTIVE_API_KEY"]
            except KeyError:
                raise ValueError(
                    "Please provide an api key or set PERSPECTIVE_API_KEY env var."
                )

        self._client = discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=api_key,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        )

    def get_toxicity_scores(self, text: str) -> Dict[str, float]:
        """Function that makes API call to Perspective to get toxicity scores across various attributes."""

        analyze_request = {
            "comment": {"text": text},
            "requestedAttributes": {att.upper(): {} for att in self.attributes},
        }

        response = self._client.comments().analyze(body=analyze_request).execute()
        try:
            return {
                att: response["attributeScores"][att.upper()]["summaryScore"]["value"]
                for att in self.attributes
            }
        except Exception as e:
            raise ValueError("Unable to parse response") from e

In [ ]:
perspective = Perspective()

In [ ]:
from typing import Tuple
from llama_index.core.bridge.pydantic import Field


def perspective_function_tool(
    text: str = Field(
        default_factory=str, description="The text to compute toxicity scores on."
    )
) -> Tuple[str, float]:
    """Returns the toxicity score of the most problematic toxic attribute."""

    scores = perspective.get_toxicity_scores(text=text)
    max_key = max(scores, key=scores.get)
    return (max_key, scores[max_key] * 100)


from llama_index.core.tools import FunctionTool

pespective_tool = FunctionTool.from_defaults(
    perspective_function_tool,
)

ModuleNotFoundError: No module named 'llama_index.core.llms.generic_utils'

In [ ]:
perspective_function_tool(text="friendly greetings from python")

## Build `InstrospectiveAgentWorker`

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.core.agent.introspective.step import IntrospectiveAgentWorker
from llama_index.core.agent.function_calling.step import FunctionCallingAgentWorker

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

### Define `IntrospectiveAgentWorker`

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgentWorker

main_worker = OpenAIAgentWorker.from_tools(
    tools=[], llm=OpenAI("gpt-4-turbo-preview"), verbose=True
)

In [ ]:
from llama_index.agent.critic import CriticAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker

critiquer = FunctionCallingAgentWorker.from_tools(
    tools=[pespective_tool], llm=OpenAI("gpt-3.5-turbo")
)

critic = CriticAgentWorker.from_args(
    critique_agent_worker=critiquer,
    critique_template="Please provide a toxicity critique for the following text:\n {input_str}",
    verbose=True,
)

In [ ]:
introspective_agent = IntrospectiveAgentWorker.from_args(
    reflective_agent_worker=critic,
    # main_agent_worker=main_worker,
    verbose=True,
).as_agent(
    chat_history=[
        ChatMessage(
            content="You are an assistant that generates safer versions of potentially toxic, user-supplied text.",
            role=MessageRole.SYSTEM,
        )
    ]
)

In [ ]:
harmful_text = (
    "Those individuals who publicly tout eating no meat, "
    "especially when their stated reason has to do with caring "
    "about animals, are some of the most hypocritical people I’ve "
    "ever met. One vegan acquaintance routinely beat her"
)
response = introspective_agent.chat(harmful_text)

Added user message to memory: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Running step f4d2ea72-905c-4c2a-9581-fd2b7eabf66b. Step input: Please provide a toxicity critique for the following text:
 Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Running step 42bb434a-c1df-4e69-86cb-74a4a8070d2f. Step input: None
Critique: The text contains an insult with a toxicity score of 50.73, indicating a high level of toxicity in the language used.
Correction: Individuals who advocate for a meat-free diet, particularly when motivated by animal welfare, often hold strong and sometimes controversial opinions. It's important to engage in respectful 

In [ ]:
response.response

"Here is a corrected version of the input.\nIndividuals who advocate for a meat-free diet, particularly when motivated by animal welfare, often hold strong and sometimes controversial opinions. It's important to engage in respectful discussions around such lifestyle choices."

In [ ]:
response.sources

[ToolOutput(content="('insult', 50.72643)", tool_name='perspective_function_tool', raw_input={'args': ('Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her',), 'kwargs': {}}, raw_output=('insult', 50.72643), is_error=False),
 ToolOutput(content="('toxicity', 1.7215505)", tool_name='perspective_function_tool', raw_input={'args': ("Individuals who advocate for a meat-free diet, particularly when motivated by animal welfare, often hold strong and sometimes controversial opinions. It's important to engage in respectful discussions around such lifestyle choices.",), 'kwargs': {}}, raw_output=('toxicity', 1.7215505), is_error=False)]

In [ ]:
introspective_agent.memory.chat_store.store

{'chat_history': [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are an assistant that generates safer versions of potentially toxic, user-supplied text.', additional_kwargs={}),
  ChatMessage(role=<MessageRole.USER: 'user'>, content='Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her', additional_kwargs={}),
  ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her', additional_kwargs={}),
  ChatMessage(role=<MessageRole.USER: 'user'>, content='The text contains an insult with a toxicity score of 50.73, indicating a high level of toxicity in the language used.', additional_k